<a href="https://colab.research.google.com/github/agrigoridou/Tokenization-Zipf-s-Law-N-gram-Models/blob/main/%CE%92_N_gram_Language_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk numpy

In [4]:
import nltk
nltk.download('treebank')


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [6]:
import nltk
from nltk.corpus import treebank
from nltk import FreqDist
from nltk.util import ngrams
from collections import defaultdict
import math

# Φόρτωση δεδομένων
train_files = treebank.fileids()[:150]
test_files = treebank.fileids()[150:]

# Συνάρτηση για δημιουργία των bigrams/trigrams
def generate_ngrams(corpus, n):
    ngrams_list = []
    for fileid in corpus:
        sents = treebank.sents(fileid)
        for sent in sents:
            # Προσθήκη των ειδικών tokens <BOS> και <EOS>
            sent = ['<BOS>'] + sent + ['<EOS>']
            ngrams_list.extend(list(ngrams(sent, n)))
    return ngrams_list

# Δημιουργία bigrams και trigrams
bigrams_train = generate_ngrams(train_files, 2)
trigrams_train = generate_ngrams(train_files, 3)

# Υπολογισμός συχνοτήτων για bigrams/trigrams
bigram_freq = FreqDist(bigrams_train)
trigram_freq = FreqDist(trigrams_train)

# Υπολογισμός λεξιλογίου
vocab = set(word for sent in treebank.sents() for word in sent)
vocab_size = len(vocab)

# Συνάρτηση για υπολογισμό πιθανότητας με add-k smoothing
def add_k_smoothing(freq_dist, n_grams, k=1):
    total_ngrams = sum(freq_dist.values())
    prob_dist = defaultdict(lambda: k / (total_ngrams + k * vocab_size))

    for ngram in freq_dist:
        prob_dist[ngram] = (freq_dist[ngram] + k) / (total_ngrams + k * vocab_size)

    return prob_dist

# Δημιουργία του bigram και trigram μοντέλου με add-k smoothing (k=1)
bigram_prob = add_k_smoothing(bigram_freq, bigrams_train, k=1)
trigram_prob = add_k_smoothing(trigram_freq, trigrams_train, k=1)


#Υπολογισμός του Perplexity

In [7]:
# Συνάρτηση για υπολογισμό perplexity
def perplexity(prob_dist, ngrams_test, n):
    log_prob_sum = 0
    N = len(ngrams_test)
    for ngram in ngrams_test:
        prob = prob_dist[ngram] if ngram in prob_dist else 1e-10  # Για να αποφύγουμε το μηδέν
        log_prob_sum += math.log(prob)
    return math.exp(-log_prob_sum / N)

# Δημιουργία bigrams και trigrams από τα κείμενα αξιολόγησης
bigrams_test = generate_ngrams(test_files, 2)
trigrams_test = generate_ngrams(test_files, 3)

# Υπολογισμός perplexity για bigrams και trigrams
bigram_perplexity = perplexity(bigram_prob, bigrams_test, 2)
trigram_perplexity = perplexity(trigram_prob, trigrams_test, 3)

print(f"Perplexity για bigrams (k=1): {bigram_perplexity}")
print(f"Perplexity για trigrams (k=1): {trigram_perplexity}")


Perplexity για bigrams (k=1): 5359925.847572324
Perplexity για trigrams (k=1): 565554787.2016815


#Μετατροπή Κειμένων σε Πεζά και Αξιολόγηση


In [8]:
# Συνάρτηση για μετατροπή των κειμένων σε πεζά
def to_lowercase(corpus):
    return [[word.lower() for word in sent] for sent in corpus]

# Μετατροπή των κειμένων σε πεζά
train_lower = to_lowercase(generate_ngrams(train_files, 1))
test_lower = to_lowercase(generate_ngrams(test_files, 1))

# Επαναλάβετε την εκπαίδευση και υπολογισμό perplexity για τα κείμενα σε πεζά


#Αντικατάσταση Ψηφίων με το ‘#’ και Αξιολόγηση

In [9]:
# Συνάρτηση για αντικατάσταση ψηφίων με το '#'
def replace_digits_with_hash(corpus):
    return [[('#' if word.isdigit() else word) for word in sent] for sent in corpus]

# Αντικατάσταση ψηφίων στα κείμενα
train_no_digits = replace_digits_with_hash(generate_ngrams(train_files, 1))
test_no_digits = replace_digits_with_hash(generate_ngrams(test_files, 1))

# Επαναλάβετε την εκπαίδευση και υπολογισμό perplexity για τα κείμενα χωρίς ψηφία


#Δημιουργία Νέων Προτάσεων


In [10]:
import random

# Συνάρτηση για τη δημιουργία νέων προτάσεων
def generate_sentence(prob_dist, n_gram_size, max_len=20):
    sentence = ['<BOS>']
    for _ in range(max_len - 1):
        candidates = [ngram for ngram in prob_dist if ngram[:n_gram_size-1] == tuple(sentence[-(n_gram_size-1):])]
        if not candidates:
            break
        next_word = random.choices(candidates, weights=[prob_dist[ngram] for ngram in candidates])[0][n_gram_size-1]
        sentence.append(next_word)
        if next_word == '<EOS>':
            break
    return sentence

# Δημιουργία νέων προτάσεων
new_sentence = generate_sentence(bigram_prob, 2)
print("Generated sentence:", ' '.join(new_sentence))


Generated sentence: <BOS> When you 'd see more than a co-owner of the hazards to 753 on whether the S&P index arbitrage
